# Training Models in AI Platform Training

## 1. Local Python Runner

In [31]:
import os
import numpy as np
PROJECT = 'crazy-hippo-01' 
BUCKET = 'crazy-hippo-01' 
REGION = 'us-central1' 
# for bash
os.environ['PROJECT'] = PROJECT
os.environ['BUCKET'] = BUCKET
os.environ['REGION'] = REGION
os.environ['TFVERSION'] = '2.1'

In [32]:
%%bash
python3 -W ignore -m trainer.task \
    --x_train_data_paths='gs://crazy-hippo-01/dataset/x_train.csv' \
    --y_train_data_paths='gs://crazy-hippo-01/dataset/y_train.csv' \
    --x_eval_data_paths='gs://crazy-hippo-01/dataset/x_val.csv'  \
    --y_eval_data_paths='gs://crazy-hippo-01/dataset/y_val.csv'  \
    --epochs=5 \
    --train_batch_size=32 \
    --output_dir=gs://$BUCKET/ai-platform/ml_engine_training \
    --num_eval_examples=100


Epoch 1/5
122/122 [==============================] - 2s 18ms/step - loss: 0.4853 - acc: 0.7705 - val_loss: 0.4087 - val_acc: 0.8059
Epoch 2/5
122/122 [==============================] - 1s 12ms/step - loss: 0.4113 - acc: 0.8066 - val_loss: 0.3835 - val_acc: 0.8159
Epoch 3/5
122/122 [==============================] - 1s 12ms/step - loss: 0.3796 - acc: 0.8248 - val_loss: 0.3736 - val_acc: 0.8228
Epoch 4/5
122/122 [==============================] - 2s 16ms/step - loss: 0.3654 - acc: 0.8286 - val_loss: 0.3686 - val_acc: 0.8194
Epoch 5/5
20/20 [==============================] - 0s 4ms/step - loss: 0.3532 - acc: 0.8359
Test Loss: 0.3531816005706787 Test Accuracy: 0.8359375
Model Saved in gs://crazy-hippo-01/ai-platform/ml_engine_training/export/savedmodel


Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


## 2. Run local using GCLOUD

In [4]:
! gcloud config set ml_engine/local_python $(which python3)

Updated property [ml_engine/local_python].


In [33]:
%%bash
gcloud ai-platform local train \
    --module-name=trainer.task \
    --package-path=${PWD}/trainer \
    -- \
    --x_train_data_paths="gs://crazy-hippo-01/dataset/x_train.csv" \
    --y_train_data_paths="gs://crazy-hippo-01/dataset/y_train.csv" \
    --x_eval_data_paths="gs://crazy-hippo-01/dataset/x_val.csv"  \
    --y_eval_data_paths="gs://crazy-hippo-01/dataset/y_val.csv"  \
    --epochs=5 \
    --train_batch_size=32 \
    --output_dir=gs://$BUCKET/ai-platform/ml_engine_training \
    --num_eval_examples=100 

Epoch 1/5
122/122 [==============================] - 3s 23ms/step - loss: 0.5060 - acc: 0.7582 - val_loss: 0.4176 - val_acc: 0.8075
Epoch 2/5
122/122 [==============================] - 1s 11ms/step - loss: 0.4126 - acc: 0.8094 - val_loss: 0.3826 - val_acc: 0.8175
Epoch 3/5
122/122 [==============================] - 1s 11ms/step - loss: 0.3854 - acc: 0.8210 - val_loss: 0.3708 - val_acc: 0.8194
Epoch 4/5
122/122 [==============================] - 1s 12ms/step - loss: 0.3679 - acc: 0.8284 - val_loss: 0.3662 - val_acc: 0.8234
Epoch 5/5
20/20 [==============================] - 0s 4ms/step - loss: 0.3497 - acc: 0.8313
Test Loss: 0.349662184715271 Test Accuracy: 0.831250011920929
Model Saved in gs://crazy-hippo-01/ai-platform/ml_engine_training/export/savedmodel


Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


## 3. Submit job to AI Platform Training

In [34]:
%%bash
OUTDIR=gs://${BUCKET}/ai-platform/ml_engine_training/earnings
JOBNAME=earnings_$(date -u +%y%m%d_%H%M%S)

gcloud ai-platform jobs submit training $JOBNAME \
    --region=$REGION \
    --module-name=trainer.task \
    --package-path=${PWD}/trainer \
    --job-dir=$OUTDIR \
    --staging-bucket=gs://$BUCKET \
    --scale-tier=BASIC \
    --runtime-version=$TFVERSION \
    --python-version=3.7 \
    -- \
    --x_train_data_paths="gs://crazy-hippo-01/dataset/x_train.csv" \
    --y_train_data_paths="gs://crazy-hippo-01/dataset/y_train.csv" \
    --x_eval_data_paths="gs://crazy-hippo-01/dataset/x_val.csv"  \
    --y_eval_data_paths="gs://crazy-hippo-01/dataset/y_val.csv"  \
    --epochs=5 \
    --train_batch_size=32 \
    --output_dir=$OUTDIR \
    --num_eval_examples=100 

jobId: earnings_210209_080602
state: QUEUED


Job [earnings_210209_080602] submitted successfully.
Your job is still active. You may view the status of your job with the command

  $ gcloud ai-platform jobs describe earnings_210209_080602

or continue streaming the logs with the command

  $ gcloud ai-platform jobs stream-logs earnings_210209_080602
